In [1]:
# Basic UI for 3D-Printed Syringe Pumps

# ===== IMPORTS =====
import sys
import os
import time
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from syringe_pump_api import SyringePumpController

# ===== HARDWARE COMMUNICATION =====

def find_pump_port():
    """Try to find the correct COM port for the syringe pump"""
    import serial.tools.list_ports
    ports = list(serial.tools.list_ports.comports())
    for port in ports:
        if 'USB' in port.description.upper() or 'CH340' in port.description.upper():
            return port.device
    return None

def cleanup():
    """Cleanup function to close serial connection on exit"""
    if hasattr(update_pump_settings, 'pump_controller'):
        try:
            update_pump_settings.pump_controller.close()
            print("✅ Cleaned up pump controller connection")
        except Exception as e:
            print(f"⚠️ Error during cleanup: {e}")

# Register cleanup function
import atexit
atexit.register(cleanup)

# ===== MAIN UI LAYOUT =====
def create_ui():
    
    # Create tab container
    tab = widgets.Tab()
    
    # Create output containers
    status_output = widgets.Output()
    
    # ===== SYRINGE PUMPS TAB =====
    # Pump selection and controls
    pump_select = widgets.Dropdown(options=['A', 'B', 'C', 'D'], value='A', description='Pump:')
    flow = widgets.FloatSlider(min=0, max=3000, step=1, value=1000, description='Flow:')
    diameter = widgets.FloatSlider(min=1, max=50, step=0.01, value=8.17, description='Diameter (mm):',style={'description_width': 'initial'})
    direction = widgets.ToggleButtons(options=['Infuse', 'Withdraw'], description='Direction:')
    state_btn = widgets.ToggleButton(description='▶ Start Pump', button_style='success')
    unit_select = widgets.Dropdown(options=['UL/MIN', 'UL/HR', 'ML/MIN', 'ML/HR'], value='UL/HR', description='Unit:')
    gearbox = widgets.Dropdown(options=['1:1', '25:1', '100:1'], value='1:1', description='Gearbox:')
    microstep = widgets.Dropdown(options=['1/8', '1/16', '1/32', '1/64'], value='1/16', description='Microstep:')
    threadrod = widgets.Dropdown(options=['1-START', '4-START'], value='1-START', description='Thread Rod:')
    refresh_btn = widgets.Button(description='🔄 Refresh Port', button_style='info')
    pump_status = widgets.Output(layout=widgets.Layout(
        width='300px',  
        height='200px',  
        overflow_y='auto',  
        border='1px solid #ddd',  
        padding='10px',  
        margin='10px 0'  
    ))
    emergency_btn = widgets.Button(
        description='EMERGENCY STOP',
        button_style='warning',
        icon='exclamation-triangle'
    )
    close_btn = widgets.Button(
    description='🔌 Close Port',
    button_style='warning',
    tooltip='Close the serial port connection'
    )
    
    def on_refresh_clicked(b):
        if hasattr(update_pump_settings, 'pump_controller'):
            try:
                update_pump_settings.pump_controller.close()
                delattr(update_pump_settings, 'pump_controller')
            except:
                pass
        update_pump_settings()
    
    refresh_btn.on_click(on_refresh_clicked)

    def on_close_clicked(b):
        if hasattr(update_pump_settings, 'pump_controller'):
            try:
                update_pump_settings.pump_controller.close()
                delattr(update_pump_settings, 'pump_controller')
                with pump_status:
                    clear_output()
                    print("✅ Port closed successfully")
            except Exception as e:
                with pump_status:
                    clear_output()
                    print(f"❌ Error closing port: {e}")
        else:
            with pump_status:
                clear_output()
                print("⚠️ No active port to close")
    
    close_btn.on_click(on_close_clicked)

    def update_pump_ui_from_config(pump_id):
        """Update UI elements with current pump configuration"""
        if not hasattr(update_pump_settings, 'pump_controller'):
            with pump_status:
                clear_output()
                print("❌ Pump controller not initialized")
            return
            
        try:
            pump = pump_id
            ctrl = update_pump_settings.pump_controller
            
            # Update UI elements
            flow.value = ctrl.get_flow(pump)
            diameter.value = ctrl.get_diameter(pump)
            
            current_dir = ctrl.get_direction(pump)
            direction.value = 'Infuse' if current_dir == 1 or (isinstance(current_dir, str) and current_dir.upper() == 'INFUSE') else 'Withdraw'
            
            unit_select.value = ctrl.get_unit(pump)
            gearbox.value = ctrl.get_gearbox(pump)
            microstep.value = ctrl.get_microstep(pump)
            threadrod.value = ctrl.get_threadrod(pump)
            state_btn.value = ctrl.get_state(pump)
            state_btn.description = '⏹ Stop Pump' if state_btn.value else '▶ Start Pump'
            
            with pump_status:
                clear_output()
                print(f"✅ Loaded config for pump {pump}")
                
        except Exception as e:
            with pump_status:
                clear_output()
                print(f"❌ Error loading config: {e}")
                import traceback
                traceback.print_exc()

    def on_pump_select(change):
        """Handle pump selection change"""
        if change['name'] == 'value':
            update_pump_ui_from_config(change['new'])

    pump_select.observe(on_pump_select, names='value')

    def create_parameter_handler(param_name, setter_func):
        def handler(change):
            if change['name'] == 'value' and hasattr(update_pump_settings, 'pump_controller'):
                try:
                    pump = pump_select.value
                    value = change['new']
                    getattr(update_pump_settings.pump_controller, setter_func)(pump, value)
                    with pump_status:
                        clear_output()
                        print(f"✅ Updated {param_name} to {value}")
                except Exception as e:
                    with pump_status:
                        clear_output()
                        print(f"❌ Error updating {param_name}: {e}")
        return handler

    # Set up parameter change handlers
    flow.observe(create_parameter_handler('flow', 'set_flow'), 'value')
    diameter.observe(create_parameter_handler('diameter', 'set_diameter'), 'value')
    direction.observe(lambda c: create_parameter_handler('direction', 'set_direction')({
        'name': 'value', 
        'new': 'INFUSE' if c['new'] == 'Infuse' else 'WITHDRAW'
    }), 'value')
    unit_select.observe(create_parameter_handler('unit', 'set_unit'), 'value')
    gearbox.observe(create_parameter_handler('gearbox', 'set_gearbox'), 'value')
    microstep.observe(create_parameter_handler('microstep', 'set_microstep'), 'value')
    threadrod.observe(create_parameter_handler('thread rod', 'set_threadrod'), 'value')

    # State button
    def on_state_click(change):
        if hasattr(update_pump_settings, 'pump_controller'):
            try:
                pump = pump_select.value
                update_pump_settings.pump_controller.set_state(pump, change['new'])
                change['owner'].description = '⏹ Stop Pump' if change['new'] else '▶ Start Pump'
                with pump_status:
                    clear_output()
                    print(f"✅ Pump {'started' if change['new'] else 'stopped'}")
            except Exception as e:
                with pump_status:
                    clear_output()
                    print(f"❌ Error: {e}")

    state_btn.observe(on_state_click, 'value')

    # Emergency stop button
    def emergency_stop(btn):
        """Immediately stop all pumps"""
        if not hasattr(update_pump_settings, 'pump_controller'):
            with pump_status:
                print("❌ No pump controller connected")
            return
        
        try:
            ctrl = update_pump_settings.pump_controller
            # Stop all pumps (A, B, C, D)
            for pump in ['A', 'B', 'C', 'D']:
                try:
                    ctrl.set_state(pump, 'STOP')
                except Exception as e:
                    with pump_status:
                        print(f"⚠️ Error stopping pump {pump}: {e}")
            
            # Update UI
            state_btn.value = False
            state_btn.description = '▶ Start Pump'
            state_btn.button_style = 'success'
            
            with pump_status:
                print("🛑 EMERGENCY STOP: All pumps have been stopped")
        except Exception as e:
            with pump_status:
                print(f"❌ Emergency stop failed: {e}")

    emergency_btn.on_click(emergency_stop)

    def update_pump_settings(change=None):
        pump = pump_select.value
        try:
            if not hasattr(update_pump_settings, 'pump_controller'):
                port = find_pump_port()
                if port is None:
                    with pump_status:
                        print("❌ Could not find syringe pump. Please check the connection.")
                    return
                try:
                    update_pump_settings.pump_controller = SyringePumpController(port)
                    with pump_status:
                        print(f"✅ Connected to syringe pump on {port}")
                except serial.SerialException as e:
                    with pump_status:
                        print("🔌 Disconnected from pump or restart kernel")
                    return
                except Exception as e:
                    with pump_status:
                        print(f"❌ Failed to initialize pump: {e}")
                    return
            
            try:
                update_pump_settings.pump_controller.set_flow(pump, flow.value)
                update_pump_settings.pump_controller.set_diameter(pump, diameter.value)
                update_pump_settings.pump_controller.set_direction(pump, direction.value.lower())
                update_pump_settings.pump_controller.set_unit(pump, unit_select.value)
                update_pump_settings.pump_controller.set_gearbox(pump, gearbox.value)
                update_pump_settings.pump_controller.set_microstep(pump, microstep.value)
                update_pump_settings.pump_controller.set_threadrod(pump, threadrod.value)
                
                with pump_status:
                    clear_output()
                    print("✅ Pump settings updated")
                    
            except Exception as e:
                with pump_status:
                    print(f"❌ Failed to update pump settings: {e}")

            if not hasattr(update_pump_settings, 'initialized'):
                # Load initial configuration for the first pump
                update_pump_ui_from_config(pump_select.value)
                update_pump_settings.initialized = True

        except Exception as e:
            with pump_status:
                print(f"❌ Unexpected error: {e}")
    
    def toggle_pump(change):
        if not hasattr(update_pump_settings, 'pump_controller'):
            with pump_status:
                print("❌ Please refresh the port first")
            state_btn.value = False
            return
            
        pump = pump_select.value
        try:
            if change['new']:  # Start pump
                update_pump_settings.pump_controller.set_state(pump, 'RUN')
                state_btn.description = '⏹ Stop Pump'
                state_btn.button_style = 'danger'
                with pump_status:
                    print(f"▶ Pump {pump} started at {flow.value} {unit_select.value}")
            else:  # Stop pump
                update_pump_settings.pump_controller.set_state(pump, 'STOP')
                state_btn.description = '▶ Start Pump'
                state_btn.button_style = 'success'
                with pump_status:
                    print(f"⏹ Pump {pump} stopped")
        except Exception as e:
            with pump_status:
                print(f"❌ Error controlling pump: {e}")
            state_btn.value = False
    
    # ===== LAYOUT =====

    pump_basic = widgets.VBox([
        widgets.HTML("<h3>Basic Controls</h3>"),
        pump_select,
        unit_select,
        flow,
        diameter,
        direction,
        widgets.Label(''),   # Empty label for spacing
        widgets.HBox([  # Put both buttons in an HBox
            state_btn,
            emergency_btn
        ], layout=widgets.Layout(justify_content='space-between'))   
    ], layout=widgets.Layout(margin='0 20px 0 0'))  # Right margin for spacing
    
    pump_advanced = widgets.VBox([
        widgets.HTML("<h3>Advanced Settings</h3>"),
        gearbox,
        microstep,
        threadrod
    ], layout=widgets.Layout(margin='0 20px 0 0'))

    pump_comm = widgets.VBox([
        widgets.HTML("<h3>Status</h3>"),
        widgets.HBox([refresh_btn, close_btn]),
        pump_status
    ], layout=widgets.Layout(
        min_width='350px',  # Ensure enough width for the status box
        margin='0 0 0 20px'  # Left margin to separate from other sections
    ))
    
    # Create a horizontal layout with spacing
    pumps_tab = widgets.HBox([
        pump_basic,
        pump_advanced,
        pump_comm
    ])
    
    # Set up tabs
    tab.children = [pumps_tab]
    tab.titles = ['Syringe Pumps']
    
    # Display UI
    display(tab)
    display(status_output)
    
    # ===== EVENT HANDLERS =====
    state_btn.observe(toggle_pump, 'value')

In [2]:
# Start the UI
create_ui()

Output()